
# Data wrangling 
#WeRateDogs


## Table of Contents:

1. Introduction
* Gathering data
* Assessing data
* Cleaning data
* Storing
* Visualizing


## Introduction
The dataset that is wrangled is the tweet archive of Twitter user @dog_rateThe purpose of this project is to put in practice what I learned in data wrangling data section from Udacity Data Analysis Nanodegree program. 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
import requests
import time
import json
import re

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Gathering Data


 twitter_archive_enhanced.csv has been downloaded manually

The tweet image predictions, i.e., what breed of dog is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv



In [3]:
# import dataset
PATH = ""
!ls {PATH}

sample_data  twitter-archive-enhanced.csv  twitter_archive_master.csv


In [62]:
df_archive = pd.read_csv(f'{PATH}twitter-archive-enhanced.csv', low_memory=False)
#tarchive.set_index('tweet_id', inplace=True)
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [91]:
image_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
r = requests.get(image_url)

with open(image_url.split('/')[-1], mode='wb') as file:
    file.write(r.content)

df_img = pd.read_csv('image-predictions.tsv', sep='\t')
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [0]:
# Twitter API setup

consumer_key = 'EY2COV3DiKnHiz5HPtfmZuONL'
consumer_secret = '0ibe93PYLQFcxzeyVjZutZKYX9MUUD12eHz8oS1tinKOAIHPVJ'
access_token = '1227329906352312323-saB80W97pWGTEsl3UdGXByI6cPXHOG'
access_secret = 'ejIifnez1uzjOz3CnsQ5SqDDtLZVwJXv1UWuJhuD53pwP'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth)
api = tweepy.API(auth, 
                 parser=tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify = True)


In [7]:
# For loop which will add each available tweet json to df_list

api_data = []

exceptions = []

tweet_ids = df_tarchive.tweet_id
# Calculate the time of execution
start = time.time()

for tweet_id in tweet_ids:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        favorites = tweet['favorite_count'] # How many favorites the tweet had
        retweets = tweet['retweet_count'] # Count of the retweet
        user_followers = tweet['user']['followers_count'] # How many followers the user had
        user_favourites = tweet['user']['favourites_count'] # How many favorites the user had
        date_time = tweet['created_at'] # The date and time of the creation
        
        api_data.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    except Exception as e:
        print(str(tweet_id)+ " _ " + str(e))
        exceptions.append(tweet_id)
# Calculate the time of excution
end = time.time()
print(end - start)

888202515573088257 _ [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921 _ [{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937 _ [{'code': 144, 'message': 'No status found with that ID.'}]
872261713294495745 _ [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329 _ [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744 _ [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320 _ [{'code': 144, 'message': 'No status found with that ID.'}]
856602993587888130 _ [{'code': 144, 'message': 'No status found with that ID.'}]
851953902622658560 _ [{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705 _ [{'code': 144, 'message': 'No status found with that ID.'}]
844704788403113984 _ [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648 _ [{'code': 144, 'message': 'No status found with that ID.'}]
837366284874571778 _ [{'code

Rate limit reached. Sleeping for: 590


754011816964026368 _ [{'code': 144, 'message': 'No status found with that ID.'}]
680055455951884288 _ [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 592


2001.3314583301544


In [31]:
# showing exception errrors
exceptions

[888202515573088257,
 873697596434513921,
 872668790621863937,
 872261713294495745,
 869988702071779329,
 866816280283807744,
 861769973181624320,
 856602993587888130,
 851953902622658560,
 845459076796616705,
 844704788403113984,
 842892208864923648,
 837366284874571778,
 837012587749474308,
 829374341691346946,
 827228250799742977,
 812747805718642688,
 802247111496568832,
 779123168116150273,
 775096608509886464,
 771004394259247104,
 770743923962707968,
 759566828574212096,
 754011816964026368,
 680055455951884288]

In [32]:
# length of fetched data and errors
len(api_data), len(exceptions)

(2331, 25)

In [33]:
# checking fecthed data
api_data[2]

{'date_time': Timestamp('2017-07-31 00:18:03+0000', tz='UTC'),
 'favorites': 23769,
 'retweets': 3824,
 'tweet_id': 891815181378084864,
 'user_favourites': 144543,
 'user_followers': 8662531}

In [0]:
df_1 = pd.DataFrame(api_data, columns = ['tweet_id', 'favorites', 'retweets', 'user_followers', 'user_favourites', 'date_time'])

In [52]:
# import tweet.txt into dataframe
#df_tweet = pd.read_csv('tweet.txt', encoding='utf-8')
df_1.head()

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,36614,7801,8662531,144543,2017-08-01 16:23:56+00:00
1,892177421306343426,31549,5776,8662531,144543,2017-08-01 00:17:27+00:00
2,891815181378084864,23769,3824,8662531,144543,2017-07-31 00:18:03+00:00
3,891689557279858688,39937,7959,8662531,144543,2017-07-30 15:58:51+00:00
4,891327558926688256,38136,8604,8662531,144543,2017-07-29 16:00:24+00:00


In [53]:
# assesing data
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
tweet_id           2331 non-null int64
favorites          2331 non-null int64
retweets           2331 non-null int64
user_followers     2331 non-null int64
user_favourites    2331 non-null int64
date_time          2331 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5)
memory usage: 109.4 KB


### Gathering Summary
- Gathered data from an existing file
- Gathered data programatically using requests
- Gathered data from Twitter API using Tweepy into JSON format

# Assesing Data

In [64]:
df_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [65]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [66]:
# descriptive stats
df_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [67]:
unique_count=df_archive.name.nunique()
unique_count

957

In [68]:
duplicate=df_archive.duplicated().sum()
duplicate

0

In [69]:
counting_null=df_archive.name.isnull().sum()
counting_null

0

In [71]:
value_rating_denominator=df_archive.rating_denominator.value_counts()
value_rating_denominator
# 10 has the highest frequency

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [73]:
num_entries=df_archive.source.value_counts()
num_entries

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [75]:
df_tarchive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [72]:
counting=df_archive.isnull().sum()
counting

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [78]:
#rating_numerator that don't follow pattern
rating_numerator_pattern=df_archive.query('rating_numerator > 20').shape[0]
rating_numerator_pattern


24

In [90]:
# checking original tweets through retweeted_status_id which would be null
null_retweets_id=df_archive[df_archive.retweeted_status_id.isnull()].shape[0]
null_retweets_id

2175

In [86]:
# unusual names 

unusual_name=df_tarchive[df_tarchive.name.apply(len)<3].shape[0]
unusual_name

79

In [92]:
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [95]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [96]:
df_img.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [0]:
df_img.img_num.value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

In [98]:
df_img.jpg_url.unique()

array(['https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
       'https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg',
       'https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg', ...,
       'https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg',
       'https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg',
       'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg'], dtype=object)

In [100]:
df_img.jpg_url.duplicated().sum()

66

In [101]:
df_img.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [0]:
# checkinng rows just to make sure if we have equal rows to df_archive
df_images.shape

(2075, 12)

In [103]:
df_images['tweet_id'].duplicated().sum()

0

In [105]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
tweet_id           2331 non-null int64
favorites          2331 non-null int64
retweets           2331 non-null int64
user_followers     2331 non-null int64
user_favourites    2331 non-null int64
date_time          2331 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(5)
memory usage: 109.4 KB


In [106]:
df_tweets.isnull().sum()

tweet_id           0
favorites          0
retweets           0
user_followers     0
user_favourites    0
date_time          0
dtype: int64

#### Quality Issues:-
- df_archive
    1. Drop unnecessary columns from df_tarchive_copy related to retweet
    2. Removing columns and rows that we don't need. Need only original tweets that have images
    3. Change datatype of TimeStamp into DateTime.
    4. Create 1 column for image prediction and 1 column for confidence level.
    5. Removing unnecessary columns from final dataset 
    6. Sources are difficult to read. Clean the content of source column.
    7. Replacing Null to NaN in all column
    8. Name that are not valid names.
    9. Numerator and Denominator column has invalid values. 
    
- df_images
    10. Missing values from image dataset 
    11. Drop date_time from df_tweets
    12. Drop unnecessary columns
    13. Some tweets_ids have the same jpg_url. There are 66 duplicated jpg_url
    14. Some tweets are have 2 different tweet_id one redirect to the other.
    15. Drop unnecessary columns
        
    
- df_tweet
    

#### Tidiness Issues:- 
    16. Merge columns into one
    17. Tables should be part of one dataframe

## Cleaning data:

In [0]:
# make copy of our original data
df_archive_copy = df_archive.copy()
df_img_copy = df_img.copy()
df_tweets_copy = df_tweets.copy()

In [112]:
df_archive[df_archive.tweet_id == 666049248165822465]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None


### 16. All tables should be part of one dataset

In [0]:
df_merged = pd.merge(left=df_archive_copy,
                        right=df_tweets_copy,
                        left_on='tweet_id', right_on='tweet_id',
                        how='inner')

In [0]:
df_merged = df_merged.merge(df_img_copy, on = 'tweet_id', how = 'inner')

In [127]:
df_merged.shape

(2059, 33)

In [128]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059 entries, 0 to 2058
Data columns (total 33 columns):
tweet_id                      2059 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2059 non-null object
source                        2059 non-null object
text                          2059 non-null object
retweeted_status_id           72 non-null float64
retweeted_status_user_id      72 non-null float64
retweeted_status_timestamp    72 non-null object
expanded_urls                 2059 non-null object
rating_numerator              2059 non-null int64
rating_denominator            2059 non-null int64
name                          2059 non-null object
doggo                         2059 non-null object
floofer                       2059 non-null object
pupper                        2059 non-null object
puppo                         2059 non-null object
favorites                     2059 

### 1. Merge the columns
    

In [0]:
value_vars =['doggo','floofer','pupper','puppo']
id_vars = [x for x in df_merged.columns.tolist() if x not in value_vars]
df_merged = pd.melt(df_merged, 
                           id_vars = id_vars,
                            value_vars = value_vars,
                            var_name='dogs', value_name='dogs_stage'
                            )

In [0]:
# Dropping unnecssary column 'dogs' from df_archive_copy
df_merged.drop('dogs', axis=1, inplace=True)

In [131]:
# Testing that we have all the data
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8236 entries, 0 to 8235
Data columns (total 30 columns):
tweet_id                      8236 non-null int64
in_reply_to_status_id         92 non-null float64
in_reply_to_user_id           92 non-null float64
timestamp                     8236 non-null object
source                        8236 non-null object
text                          8236 non-null object
retweeted_status_id           288 non-null float64
retweeted_status_user_id      288 non-null float64
retweeted_status_timestamp    288 non-null object
expanded_urls                 8236 non-null object
rating_numerator              8236 non-null int64
rating_denominator            8236 non-null int64
name                          8236 non-null object
favorites                     8236 non-null int64
retweets                      8236 non-null int64
user_followers                8236 non-null int64
user_favourites               8236 non-null int64
date_time                     8236 n

In [0]:
#Sorting by dogs_stage we can also use groupbythen drop duplicated based on tweet_id except the last occurrence
df_merged = df_merged.sort_values('dogs_stage').drop_duplicates(subset='tweet_id', keep='last')

In [133]:
#REcheck
df_merged['dogs_stage'].value_counts()

None       1741
pupper      221
doggo        65
puppo        24
floofer       8
Name: dogs_stage, dtype: int64

In [136]:
df_merged[df_merged.tweet_id == 666049248165822465]['text']

2054    Here we have a 1949 1st generation vulpix. Enj...
Name: text, dtype: object

In [137]:
df_merged[df_merged.tweet_id == 666049248165822465]['dogs_stage']

2054    None
Name: dogs_stage, dtype: object

### 2. Removing columns and rows that we don't need. Need only original tweets that have images.

In [0]:
df_merged = df_merged[df_merged.retweeted_status_id.isnull()]

In [0]:
df_merged = df_merged.drop_duplicates()

In [140]:
# Testing that we have all the data we need
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 1982 to 7053
Data columns (total 30 columns):
tweet_id                      1987 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     1987 non-null object
source                        1987 non-null object
text                          1987 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1987 non-null object
rating_numerator              1987 non-null int64
rating_denominator            1987 non-null int64
name                          1987 non-null object
favorites                     1987 non-null int64
retweets                      1987 non-null int64
user_followers                1987 non-null int64
user_favourites               1987 non-null int64
date_time                     1987 non-

### 3. Drop unnecessary columns from df_archive_copy related to retweet

In [0]:
df_merged.drop(['retweeted_status_id',
                     'retweeted_status_user_id',
                       'retweeted_status_timestamp'],
                     axis = 1, inplace = True)

In [0]:
# checking if we have deleted the above columns
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 1982 to 7053
Data columns (total 27 columns):
tweet_id                 1987 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1987 non-null object
source                   1987 non-null object
text                     1987 non-null object
expanded_urls            1987 non-null object
rating_numerator         1987 non-null int64
rating_denominator       1987 non-null int64
name                     1987 non-null object
favorites                1987 non-null int64
retweets                 1987 non-null int64
user_followers           1987 non-null int64
user_favourites          1987 non-null int64
date_time                1987 non-null object
jpg_url                  1987 non-null object
img_num                  1987 non-null int64
p1                       1987 non-null object
p1_conf                  1987 non-null float64
p1_dog                   

In [0]:
# Drop Duplicates dog_breed
df_merged.sort_values('dogs_stage').drop_duplicates(['tweet_id'], inplace=True)

In [0]:
# checking all the dog breed 
df_merged.dogs_stage.value_counts()

None       1682
pupper      212
doggo        62
puppo        23
floofer       8
Name: dogs_stage, dtype: int64

### 4. Change datatype of TimeStamp into DateTime.

In [0]:
df_merged.timestamp = pd.to_datetime(df_merged.timestamp)

In [142]:
df_merged.dtypes

tweet_id                                    int64
in_reply_to_status_id                     float64
in_reply_to_user_id                       float64
timestamp                     datetime64[ns, UTC]
source                                     object
text                                       object
retweeted_status_id                       float64
retweeted_status_user_id                  float64
retweeted_status_timestamp                 object
expanded_urls                              object
rating_numerator                            int64
rating_denominator                          int64
name                                       object
favorites                                   int64
retweets                                    int64
user_followers                              int64
user_favourites                             int64
date_time                     datetime64[ns, UTC]
jpg_url                                    object
img_num                                     int64


### 5. Create 1 column for image prediction and 1 column for confidence level.

In [0]:
# writing an prediction_confidence function
# we are going to check if prediction is true, if yes then going to append prediction and confidence to their respective columns

pred_algo = []
conf_level = []

def pred_conf(dataframe):
    if dataframe['p1_dog'] == True:
        pred_algo.append(dataframe['p1'])
        conf_level.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        pred_algo.append(dataframe['p2'])
        conf_level.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        pred_algo.append(dataframe['p3'])
        conf_level.append(dataframe['p3_conf'])
    else:
        pred_algo.append('NaN')
        conf_level.append(0)

In [0]:
# calling pred_conf function 
df_merged.apply(pred_conf, axis = 1)
df_merged['pred_algo'] = pred_algo
df_merged['conf_level'] = conf_level

In [145]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 1982 to 7053
Data columns (total 32 columns):
tweet_id                      1987 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     1987 non-null datetime64[ns, UTC]
source                        1987 non-null object
text                          1987 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1987 non-null object
rating_numerator              1987 non-null int64
rating_denominator            1987 non-null int64
name                          1987 non-null object
favorites                     1987 non-null int64
retweets                      1987 non-null int64
user_followers                1987 non-null int64
user_favourites               1987 non-null int64
date_time                 

### 6. Removing unnecessary columns from df_merged

In [0]:
df_merged.drop(['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], axis=1, inplace=True)

In [0]:
df_merged.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'user_favourites'], axis=1, inplace=True)

In [148]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 1982 to 7053
Data columns (total 19 columns):
tweet_id                      1987 non-null int64
timestamp                     1987 non-null datetime64[ns, UTC]
source                        1987 non-null object
text                          1987 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1987 non-null object
rating_numerator              1987 non-null int64
rating_denominator            1987 non-null int64
name                          1987 non-null object
favorites                     1987 non-null int64
retweets                      1987 non-null int64
user_followers                1987 non-null int64
date_time                     1987 non-null datetime64[ns, UTC]
jpg_url                       1987 non-null object
dogs_stage                    1987 non-null object
pred_algo 

In [149]:
df_merged.head()

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,favorites,retweets,user_followers,date_time,jpg_url,dogs_stage,pred_algo,conf_level
1982,667435689202614272,2015-11-19 20:14:03+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Ermergerd 12/10 https://t.co/PQni2sjPsm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667435689...,12,10,None,298,82,8662578,2015-11-19 20:14:03+00:00,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,None,Rottweiler,0.999091
1981,667437278097252352,2015-11-19 20:20:22+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Never seen this breed before. Very pointy pup....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667437278...,10,10,None,437,227,8662578,2015-11-19 20:20:22+00:00,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,None,NaN,0.000000
1980,667443425659232256,2015-11-19 20:44:47+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Exotic dog here. Long neck. Weird paws. Obsess...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667443425...,6,10,None,755,555,8662578,2015-11-19 20:44:47+00:00,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,None,NaN,0.000000
1979,667453023279554560,2015-11-19 21:22:56+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Meet Cupcake. I would do unspeakable things fo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667453023...,11,10,Cupcake,296,85,8662577,2015-11-19 21:22:56+00:00,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,None,Labrador_retriever,0.825670
1978,667455448082227200,2015-11-19 21:32:34+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Reese and Twips. Reese protects Twips....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667455448...,7,10,Reese,184,57,8662577,2015-11-19 21:32:34+00:00,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,None,Tibetan_terrier,0.676376


### 7. Sources are difficult to read. Clean the content of source column.


In [0]:
df_merged['source'] = df_merged['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

In [151]:
df_merged.head()

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,favorites,retweets,user_followers,date_time,jpg_url,dogs_stage,pred_algo,conf_level
1982,667435689202614272,2015-11-19 20:14:03+00:00,Twitter for iPhone,Ermergerd 12/10 https://t.co/PQni2sjPsm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667435689...,12,10,None,298,82,8662578,2015-11-19 20:14:03+00:00,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,None,Rottweiler,0.999091
1981,667437278097252352,2015-11-19 20:20:22+00:00,Twitter for iPhone,Never seen this breed before. Very pointy pup....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667437278...,10,10,None,437,227,8662578,2015-11-19 20:20:22+00:00,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,None,NaN,0.000000
1980,667443425659232256,2015-11-19 20:44:47+00:00,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667443425...,6,10,None,755,555,8662578,2015-11-19 20:44:47+00:00,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,None,NaN,0.000000
1979,667453023279554560,2015-11-19 21:22:56+00:00,Twitter Web Client,Meet Cupcake. I would do unspeakable things fo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667453023...,11,10,Cupcake,296,85,8662577,2015-11-19 21:22:56+00:00,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,None,Labrador_retriever,0.825670
1978,667455448082227200,2015-11-19 21:32:34+00:00,Twitter Web Client,This is Reese and Twips. Reese protects Twips....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667455448...,7,10,Reese,184,57,8662577,2015-11-19 21:32:34+00:00,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,None,Tibetan_terrier,0.676376


In [0]:
# removing date_time column
df_merged.drop('date_time', axis = 1, inplace=True)

In [153]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 1982 to 7053
Data columns (total 18 columns):
tweet_id                      1987 non-null int64
timestamp                     1987 non-null datetime64[ns, UTC]
source                        1987 non-null object
text                          1987 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1987 non-null object
rating_numerator              1987 non-null int64
rating_denominator            1987 non-null int64
name                          1987 non-null object
favorites                     1987 non-null int64
retweets                      1987 non-null int64
user_followers                1987 non-null int64
jpg_url                       1987 non-null object
dogs_stage                    1987 non-null object
pred_algo                     1987 non-null object
conf_level             

### 8. Strings are not accuartely found in 'name' column

In [154]:
lowercase_names = []
for row in df_merged['name']:
    if row[0].islower() and row not in lowercase_names:
        lowercase_names.append(row)
print(lowercase_names)

['a', 'an', 'quite', 'the', 'such', 'getting', 'actually', 'infuriating', 'very', 'light', 'space', 'just', 'by', 'officially', 'incredibly', 'one', 'my', 'not', 'this', 'unacceptable', 'his', 'all']


In [0]:
# Replace all names that start with a lowercase letter with NaN
df_merged['name'].replace(lowercase_names, 
                        np.nan,
                       inplace = True)

# Replace all 'None's with NaN
df_merged['name'].replace('None', 
                        np.nan,
                       inplace = True)

In [156]:
df_merged.name.value_counts()

Oliver      10
Cooper      10
Charlie     10
Penny        9
Lucy         9
            ..
William      1
Vinscent     1
Dug          1
Billy        1
Hanz         1
Name: name, Length: 911, dtype: int64

### 9. Replace the Numerator and denominator ratings



In [157]:
# Obtain all text, indices, and ratings for tweets that contain a decimal 
# in the numerator of rating
ratings_with_decimals_text = []
ratings_with_decimals_index = []
ratings_with_decimals = []


for i, text in df_merged['text'].iteritems():
    if bool(re.search('\d+\.\d+\/\d+', text)):
        ratings_with_decimals_text.append(text)
        ratings_with_decimals_index.append(i)
        ratings_with_decimals.append(re.search('\d+\.\d+', text).group())

# Print the text to confirm presence of ratings with decimals        
ratings_with_decimals_text

['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
 'Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD',
 "This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",
 "This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq"]

In [158]:
# Print the index of text with decimal ratings
ratings_with_decimals_index

[40, 3497, 2607, 4721]

In [0]:
# Changeing contents of 'rating_numerator' based on info found in preceeding 2 cells
df_merged.loc[ratings_with_decimals_index[0],'rating_numerator'] = float(ratings_with_decimals[0])
df_merged.loc[ratings_with_decimals_index[1],'rating_numerator'] = float(ratings_with_decimals[1])
df_merged.loc[ratings_with_decimals_index[2],'rating_numerator'] = float(ratings_with_decimals[2])
df_merged.loc[ratings_with_decimals_index[3],'rating_numerator'] = float(ratings_with_decimals[3])

In [160]:
# Check contents of row with index 40 to ensure the rating is corrected
df_merged.loc[40]

tweet_id                                                     883482846933004288
timestamp                                             2017-07-08 00:28:19+00:00
source                                                       Twitter for iPhone
text                          This is Bella. She hopes her smile made you sm...
retweeted_status_id                                                         NaN
retweeted_status_user_id                                                    NaN
retweeted_status_timestamp                                                  NaN
expanded_urls                 https://twitter.com/dog_rates/status/883482846...
rating_numerator                                                           13.5
rating_denominator                                                           10
name                                                                      Bella
favorites                                                                 43482
retweets                                

In [0]:
# create new column with rating
df_merged['rating'] = df_merged['rating_numerator'] / df_merged['rating_denominator']

In [162]:
df_merged.head()

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,favorites,retweets,user_followers,jpg_url,dogs_stage,pred_algo,conf_level,rating
1982,667435689202614272,2015-11-19 20:14:03+00:00,Twitter for iPhone,Ermergerd 12/10 https://t.co/PQni2sjPsm,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667435689...,12.0,10,NaN,298,82,8662578,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,None,Rottweiler,0.999091,1.2
1981,667437278097252352,2015-11-19 20:20:22+00:00,Twitter for iPhone,Never seen this breed before. Very pointy pup....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667437278...,10.0,10,NaN,437,227,8662578,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,None,NaN,0.000000,1.0
1980,667443425659232256,2015-11-19 20:44:47+00:00,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667443425...,6.0,10,NaN,755,555,8662578,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,None,NaN,0.000000,0.6
1979,667453023279554560,2015-11-19 21:22:56+00:00,Twitter Web Client,Meet Cupcake. I would do unspeakable things fo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667453023...,11.0,10,Cupcake,296,85,8662577,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,None,Labrador_retriever,0.825670,1.1
1978,667455448082227200,2015-11-19 21:32:34+00:00,Twitter Web Client,This is Reese and Twips. Reese protects Twips....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667455448...,7.0,10,Reese,184,57,8662577,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,None,Tibetan_terrier,0.676376,0.7


In [163]:
df_merged.shape

(1987, 19)

## Now converting into csv and it is ready to get use

In [0]:
#Store the clean DataFrame in a CSV file
df=df_merged.to_csv('twitter_archive_master.csv', 
                 index=False, encoding = 'utf-8')